In [1]:
import numpy as np
from tensorflow import keras

In [2]:
def generate_time_series(batch_size, n_steps):
    freq1 ,freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offset2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [3]:
n_steps = 50
series = generate_time_series(10000, n_steps + 10)

train_X = series[:7500, :n_steps]
valid_X = series[7500:, :n_steps]

y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 11):
    y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]

train_y = y[:7500]
valid_y = y[7500:]

In [4]:
early_stop_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

def last_time_step_mse(y_true, y_pred):
    return keras.metrics.mean_squared_error(y_true[:, -1], y_pred[:, -1])

In [5]:
model_simpleRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(10, return_sequences=True, input_shape=[None, 1])
])

model_simpleRNN.compile(loss='mean_squared_error', optimizer='Adam', metrics=[last_time_step_mse])
model_simpleRNN.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 2s 6ms/step - loss: 0.1513 - last_time_step_mse: 0.1479 - val_loss: 0.1088 - val_last_time_step_mse: 0.1049
Epoch 2/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0943 - last_time_step_mse: 0.0884 - val_loss: 0.0808 - val_last_time_step_mse: 0.0746
Epoch 3/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0700 - last_time_step_mse: 0.0614 - val_loss: 0.0625 - val_last_time_step_mse: 0.0537
Epoch 4/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0587 - last_time_step_mse: 0.0489 - val_loss: 0.0574 - val_last_time_step_mse: 0.0501
Epoch 5/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0553 - last_time_step_mse: 0.0454 - val_loss: 0.0546 - val_last_time_step_mse: 0.0455
Epoch 6/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0538 - last_time_step_mse: 0.0438 - val_loss: 0.0535 - val_last_time_step_mse: 0.0444
Epoch 7/50
235/235 [========

In [6]:
model_deepRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model_deepRNN.compile(loss='mean_squared_error', optimizer='Adam', metrics=[last_time_step_mse])
model_deepRNN.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 4s 13ms/step - loss: 0.0665 - last_time_step_mse: 0.0568 - val_loss: 0.0471 - val_last_time_step_mse: 0.0367
Epoch 2/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0434 - last_time_step_mse: 0.0303 - val_loss: 0.0424 - val_last_time_step_mse: 0.0279
Epoch 3/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0391 - last_time_step_mse: 0.0239 - val_loss: 0.0379 - val_last_time_step_mse: 0.0221
Epoch 4/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0372 - last_time_step_mse: 0.0212 - val_loss: 0.0364 - val_last_time_step_mse: 0.0205
Epoch 5/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0357 - last_time_step_mse: 0.0193 - val_loss: 0.0345 - val_last_time_step_mse: 0.0179
Epoch 6/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0341 - last_time_step_mse: 0.0176 - val_loss: 0.0334 - val_last_time_step_mse: 0.0163
Epoch 7/50
235/235 [==

In [7]:
model_singleLSTM = keras.models.Sequential([
    keras.layers.LSTM(10, return_sequences=True, input_shape=[None,1]),
])

model_singleLSTM.compile(loss='mean_squared_error', optimizer='Adam', metrics=[last_time_step_mse])
model_singleLSTM.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 4s 12ms/step - loss: 0.0999 - last_time_step_mse: 0.0931 - val_loss: 0.0718 - val_last_time_step_mse: 0.0598
Epoch 2/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0659 - last_time_step_mse: 0.0523 - val_loss: 0.0632 - val_last_time_step_mse: 0.0510
Epoch 3/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0608 - last_time_step_mse: 0.0479 - val_loss: 0.0598 - val_last_time_step_mse: 0.0488
Epoch 4/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0581 - last_time_step_mse: 0.0456 - val_loss: 0.0574 - val_last_time_step_mse: 0.0467
Epoch 5/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0556 - last_time_step_mse: 0.0430 - val_loss: 0.0546 - val_last_time_step_mse: 0.0426
Epoch 6/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0531 - last_time_step_mse: 0.0392 - val_loss: 0.0527 - val_last_time_step_mse: 0.0395
Epoch 7/50
235/235 [==

In [9]:
model_deepLSTM = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None,1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model_deepLSTM.compile(loss='mean_squared_error', optimizer='Adam', metrics=[last_time_step_mse])
model_deepLSTM.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 8s 24ms/step - loss: 0.0757 - last_time_step_mse: 0.0613 - val_loss: 0.0536 - val_last_time_step_mse: 0.0344
Epoch 2/50
235/235 [==============================] - 5s 22ms/step - loss: 0.0467 - last_time_step_mse: 0.0257 - val_loss: 0.0416 - val_last_time_step_mse: 0.0202
Epoch 3/50
235/235 [==============================] - 5s 20ms/step - loss: 0.0380 - last_time_step_mse: 0.0172 - val_loss: 0.0367 - val_last_time_step_mse: 0.0173
Epoch 4/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0333 - last_time_step_mse: 0.0134 - val_loss: 0.0324 - val_last_time_step_mse: 0.0135
Epoch 5/50
235/235 [==============================] - 5s 22ms/step - loss: 0.0307 - last_time_step_mse: 0.0117 - val_loss: 0.0304 - val_last_time_step_mse: 0.0117
Epoch 6/50
235/235 [==============================] - 6s 24ms/step - loss: 0.0291 - last_time_step_mse: 0.0108 - val_loss: 0.0282 - val_last_time_step_mse: 0.0102
Epoch 7/50
235/235 [==

In [10]:
model_wavenet = keras.models.Sequential()
model_wavenet.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2:
    model_wavenet.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding='causal', activation='relu', dilation_rate=rate))
model_wavenet.add(keras.layers.Conv1D(filters=10, kernel_size=1))

model_wavenet.compile(loss='mean_squared_error', optimizer='Adam', metrics=[last_time_step_mse])
model_wavenet.fit(train_X, train_y, epochs=100, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/100
235/235 [==============================] - 3s 9ms/step - loss: 0.0613 - last_time_step_mse: 0.0488 - val_loss: 0.0352 - val_last_time_step_mse: 0.0213
Epoch 2/100
235/235 [==============================] - 2s 8ms/step - loss: 0.0321 - last_time_step_mse: 0.0194 - val_loss: 0.0299 - val_last_time_step_mse: 0.0181
Epoch 3/100
235/235 [==============================] - 2s 8ms/step - loss: 0.0281 - last_time_step_mse: 0.0164 - val_loss: 0.0264 - val_last_time_step_mse: 0.0147
Epoch 4/100
235/235 [==============================] - 2s 8ms/step - loss: 0.0259 - last_time_step_mse: 0.0143 - val_loss: 0.0255 - val_last_time_step_mse: 0.0142
Epoch 5/100
235/235 [==============================] - 2s 8ms/step - loss: 0.0244 - last_time_step_mse: 0.0128 - val_loss: 0.0240 - val_last_time_step_mse: 0.0118
Epoch 6/100
235/235 [==============================] - 2s 8ms/step - loss: 0.0234 - last_time_step_mse: 0.0118 - val_loss: 0.0233 - val_last_time_step_mse: 0.0117
Epoch 7/100
235/235 [=

235/235 [==============================] - 2s 8ms/step - loss: 0.0156 - last_time_step_mse: 0.0051 - val_loss: 0.0162 - val_last_time_step_mse: 0.0053
Epoch 52/100
235/235 [==============================] - 2s 8ms/step - loss: 0.0157 - last_time_step_mse: 0.0052 - val_loss: 0.0161 - val_last_time_step_mse: 0.0056
Epoch 53/100
235/235 [==============================] - 2s 8ms/step - loss: 0.0156 - last_time_step_mse: 0.0051 - val_loss: 0.0158 - val_last_time_step_mse: 0.0049
Epoch 54/100
235/235 [==============================] - 2s 8ms/step - loss: 0.0155 - last_time_step_mse: 0.0050 - val_loss: 0.0155 - val_last_time_step_mse: 0.0048
Epoch 55/100
235/235 [==============================] - 2s 8ms/step - loss: 0.0156 - last_time_step_mse: 0.0051 - val_loss: 0.0157 - val_last_time_step_mse: 0.0051
Epoch 56/100
235/235 [==============================] - 2s 8ms/step - loss: 0.0157 - last_time_step_mse: 0.0053 - val_loss: 0.0157 - val_last_time_step_mse: 0.0052
Epoch 57/100
235/235 [=======